In [ ]:
#############################################################
#                     Naver Webtton Crwaling                #
#############################################################

import requests
import csv
from bs4 import BeautifulSoup
import re
import json
import time
import argparse
import requests
import pathlib2
import openpyxl
from urllib.parse import urlparse, parse_qs

wb = openpyxl.Workbook()
sheet = wb.active

def get_commentList(title, titleId, episode_no, comment_page):
    ajax_link = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon' + \
                '&pool=cbox3&_callback=jQuery112409768039369695578_1588773317723&lang=ko&country=KR&objectId=' + \
                titleId + '_' + episode_no + '&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=' + str(comment_page) + \
                '&refresh=true&sort=new&cleanbotGrade=2&_=1588773317725'
    custom_header = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=' + titleId + '&no=' + episode_no,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'
    }
    req = requests.get(ajax_link, headers=custom_header).text
    json_req = json.loads(req[req.find("success") - 2:len(req) - 2])
    commentList = json_req['result']['commentList']
    return commentList

def save_comments(title, titleId, episode_no):
    comment_page = 1
    all_comments_info= []
    cnt=0
    num=0

    while True:
        cnt+=1
        commentList = get_commentList(title, titleId, episode_no, comment_page)
        if commentList:
            for comment in commentList:
              num+=1;
              all_comments_info=[]
              all_comments_info.append(title)
              all_comments_info.append(episode_no)
              all_comments_info.append(comment['maskedUserId'])
              all_comments_info.append(comment['contents'].replace("\n",""))
              print(str(num)+'번째 ',title,episode_no,comment['maskedUserId'],comment['contents'].replace("\n",""))
              try:
                sheet.append([title,episode_no,comment['maskedUserId'],comment['contents'].replace("\n","")])
              except:
                print("error")
        else:
            wb.save("naver_webtoon_comments.xlsx")
            return
        comment_page = comment_page + 1


sheet.append(["제목", "에피소드화", "ID", "댓글 내용"])


url_episode = 'https://comic.naver.com/webtoon/genre.nhn'
url_omnibus = 'https://comic.naver.com/webtoon/genre.nhn?genre=omnibus'
url_story = 'https://comic.naver.com/webtoon/genre.nhn?genre=story'

url_list = [url_episode, url_omnibus, url_story]
title_list = []
for i in url_list:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    temp = soup.find_all('dl')
    for j in temp:
        title_list.append(j)

cnt=0
for a in title_list:
    cnt+=1
    print(cnt, "번째 웹툰 크롤링 중...")
    webtoon_info = [cnt]
    webtoon_info.append(a.find('a').get_text())
    print("제목 : ", a.find('a').get_text())
    webtoon_info.append(a.find('dd').get_text()[1:len(a.find('dd').get_text())-1])
    print("작가 : ", a.find('dd').get_text()[1:len(a.find('dd').get_text())-1])
   
    webtoon_url =  "https://comic.naver.com" + a.find('a').get('href')
    response = requests.get(webtoon_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(webtoon_url)

    webtoon_round_url=soup.find('td',class_='title').find('a').get('href')
    print(soup.find('td',class_='title').find('a').get('href'))

    #id, 회차정보, 요일
    tId=webtoon_round_url.split('titleId=')[1].split('&')[0]
    tRound=webtoon_round_url.split('no=')[1].split('&')[0]
    tWeekday=webtoon_round_url.split('weekday=')[1].split('&')[0]
    webtoon_info.append(tId)
    webtoon_info.append(tRound)
    webtoon_info.append(tWeekday)

    print("아이디: ",tId)
    print("회차정보: ", tRound)
    print("요일: ",tWeekday)

    round_cnt=0

    for i in reversed(range((int(tRound)+1))):
      # https://comic.naver.com/webtoon/detail.nhn?titleId=733766&no=56&weekday=mon 형식
      round_cnt+=1
      webtoon_comment_url="https://comic.naver.com/webtoon/detail.nhn?"+ "titleId="+tId+"&no="+str(i)+"&weekday="+tWeekday
      save_comments(a.find('a').get_text(),tId,str(i))
      if round_cnt==10:
        break

print(cnt, "개 웹툰 크롤링 종료")
wb.save("naver_webtoon_comments.xlsx")
# print("출력 파일 : crawling_result")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1070번째  한림체육관 19 cho3**** 케넨 vs 세트 를 이길 드립은 없을듯ㅋㅋ
1071번째  한림체육관 19 baem**** 웅? 한국여자도 보는뎈ㅋㅋㅋㅋ
1072번째  한림체육관 19 annt**** 아무이유없이배댓돼기        ㅡ수정됨
1073번째  한림체육관 19 carm**** 왜 이번편은 베댓이 두개밖에 없을까
1074번째  한림체육관 19 wwew**** 스태틱 에코 vs 세트
1075번째  한림체육관 19 myna**** 짧게 느겨지는거냐 짧은거냐?
1076번째  한림체육관 19 shin**** 베댓 없어서 놀라신분?
1077번째  한림체육관 19 leej**** 이기태 지금까지 힘숨기고 있던거임?
1078번째  한림체육관 19 teot**** ㅋㅋㅋㅋㅋㅋ 전기막대기? 저걸로 싸우는거 왤케 웃기냨ㅋㅋㅋㅋㅋㅋ 개윳기넼ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 저거 믿고 싸우는거 ㅋㅋㅋㅋㅋㅋㅋㅋ
1079번째  한림체육관 19 dlwn**** 규현씨 때문에 베댓 3개밖에 없는거 실화냐.. 가슴이 웅장 해진다
1080번째  한림체육관 19 hong**** 김규현님 변태맞는거같은데여?
1081번째  한림체육관 19 ooha**** 덕순이ㅅㅂㅋㅋㅋㅋㅋㅋ나만 존나웃기냐 왜 아무도 이얘기 안함 ㅋㅋㅋㅋㅋ개웃긴데ㅋㅎㅋㅅㅋㅋㅋㅋㅋㅋㅋ
1082번째  한림체육관 19 31__**** 영하의 과거
1083번째  한림체육관 19 dora**** 사건 검색해바ㅋㅋ 좀 작작올려라 역겹다
1084번째  한림체육관 19 rlar**** 분량이...흑흑...
1085번째  한림체육관 19 skag**** 서태진 이미 애들 없이 1ㄷ1이였으면 벌써 ㄷ졌어 전영하한테 그니까 너무 흥분하고 그러지마
1086번째  한림체육관 19 jcjd**** 케넨 vs 세트를 이길만한 드립이 떠오르지 않는다 ..
1087번째  한림체육관 19 game**** 규현좌 이미 봐버렸다..ㅋㅋㅋ
1088번째  한림체육관 19 sinj**** 세번째 배댓 